# BERT Model for Cyberbullying Detection

Fine-tuning pre-trained BERT for identifying cyberbullying in social media posts

In [ ]:
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time

from src.train import clean_text
from src.utils import calculate_metrics

In [ ]:
df = pd.read_csv("../data/raw/train.csv")
print(f"Total samples: {len(df)}")

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print(f"\nLabel distribution:")
print(df[label_cols].sum())

In [ ]:
# use subset for faster training
df_sample = df.sample(n=20000, random_state=42)
print(f"Using {len(df_sample)} samples for BERT training")

In [ ]:
df_sample["clean_text"] = df_sample["comment_text"].apply(clean_text)

X_train, X_test, y_train, y_test = train_test_split(
    df_sample["clean_text"].values, df_sample[label_cols].values,
    test_size=0.2, random_state=42
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

## BERT Tokenizer and Dataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
MAX_LEN = 128

class BERTDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(self.labels[idx])
        }

In [ ]:
BATCH_SIZE = 16

train_dataset = BERTDataset(X_train, y_train, tokenizer, MAX_LEN)
test_dataset = BERTDataset(X_test, y_test, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

## BERT Classifier

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, n_classes, dropout=0.3):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        output = self.dropout(pooled_output)
        return self.fc(output)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

model = BERTClassifier(n_classes=len(label_cols), dropout=0.3).to(device)
print(f"\nModel parameters: {sum(p.numel() for p in model.parameters())}")

## Training

In [ ]:
def train_epoch_bert(model, data_loader, optimizer, criterion, device):
    model.train()
    losses = []
    
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
    
    return np.mean(losses)

def evaluate_bert(model, data_loader, criterion, device):
    model.eval()
    losses = []
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
    
    return np.mean(losses)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCEWithLogitsLoss()

n_epochs = 3
train_losses = []
test_losses = []

start_time = time.time()

for epoch in range(n_epochs):
    train_loss = train_epoch_bert(model, train_loader, optimizer, criterion, device)
    test_loss = evaluate_bert(model, test_loader, criterion, device)
    
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    print(f'Epoch {epoch+1:02d} | Train Loss: {train_loss:.3f} | Test Loss: {test_loss:.3f}')

train_time = time.time() - start_time
print(f"\nTraining time: {train_time:.2f}s")

In [ ]:
torch.save(model.state_dict(), '../outputs/bert_model.pt')

## Evaluation

In [ ]:
def get_predictions_bert(model, data_loader, device, threshold=0.5):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']
            
            outputs = model(input_ids, attention_mask)
            preds = (torch.sigmoid(outputs) > threshold).float()
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    return np.array(all_preds), np.array(all_labels)

In [ ]:
y_pred, y_true = get_predictions_bert(model, test_loader, device)
metrics = calculate_metrics(y_true, y_pred, label_cols)

print("Overall Results:")
for k, v in metrics['overall'].items():
    print(f"{k}: {v:.4f}")

print("\nPer-label F1 scores:")
for label in label_cols:
    print(f"{label}: {metrics[label]['f1']:.4f}")

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(train_losses, label='Train', marker='o')
plt.plot(test_losses, label='Test', marker='o')
plt.title('BERT Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/bert_training.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
f1_scores = [metrics[label]['f1'] for label in label_cols]

plt.figure(figsize=(10, 5))
plt.bar(label_cols, f1_scores)
plt.title('Per-label F1 Scores')
plt.ylabel('F1 Score')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/bert_f1_scores.png', dpi=150, bbox_inches='tight')
plt.show()